In [1]:
import pandas as pd
pd.set_option('display.width',100)
pd.set_option('display.max_columns',100)

In [2]:
metadata=pd.read_csv("../Datasets/HAM10000_metadata.csv")

In [6]:
metadata.info()
metadata.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10015 entries, 0 to 10014
Data columns (total 7 columns):
lesion_id       10015 non-null object
image_id        10015 non-null object
dx              10015 non-null object
dx_type         10015 non-null object
age             9958 non-null float64
sex             10015 non-null object
localization    10015 non-null object
dtypes: float64(1), object(6)
memory usage: 547.8+ KB


,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [7]:
table={k: table for k, table in zip(np.unique(metadata['dx']),metadata.groupby("dx").size())}

dx
akiec     327
bcc       514
bkl      1099
df        115
mel      1113
nv       6705
vasc      142
dtype: int64

In [38]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
x=compute_class_weight('balanced',
                       np.unique(metadata['dx']),
                       metadata['dx'].values)

In [39]:
x


array([ 4.37527304,  2.78349083,  1.30183284, 12.44099379,  1.28545758,
        0.21338021, 10.07545272])

In [43]:
from keras_preprocessing.image import ImageDataGenerator

DATA_DIR="../Datasets/HAM10000_images"


num_rows=120
num_cols=90
batch_size = 128
nb_epoch = 10
num_classes = 7

def append_ext(fn):
	return fn+".jpg"

metadata['image_id']=metadata['image_id'].apply(append_ext)

datagen=ImageDataGenerator(rotation_range=180, 
						   width_shift_range=0.15, 
						   height_shift_range=0.15,
						   horizontal_flip= True, 
						   vertical_flip=True,
						   validation_split=0.2)

train_generator=datagen.flow_from_dataframe(
											dataframe=metadata,
											directory=DATA_DIR,
											x_col="image_id",
											y_col="dx",
											subset="training",
											batch_size=batch_size,
											seed=169,
											shuffle=True,
											class_mode="categorical",
											target_size=(num_rows,num_cols))

Found 8012 validated image filenames belonging to 7 classes.


In [51]:
train_generator.n

8012